In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
fmnist_data = keras.datasets.fashion_mnist

In [4]:
(x_train, y_train), (x_test, y_test) = fmnist_data.load_data()

In [5]:
x_train.shape

(60000, 28, 28)